
# SixtyFour AI — Find Email

This notebook is your guide to using the Sixtyfour API `find-email`.

The `find-email` endpoint finds an email address for a lead based on the information you provide. It searches relevant sources and can validate deliverability. You can optionally restrict results to **company domains** and control whether to use **bruteforce** heuristics.

**Tip:** You can change the lead in the parameters at the top. For best results, include the **name** and **company** (and optionally a strong identifier such as **LinkedIn**).



## Prerequisites & Secrets

### Get your SixtyFour API key
1) Sign in: https://app.sixtyfour.ai/login and get your $25 in credits
2) Create an organization → **Keys** → **Create new key** → Copy the key.

### If you’re in Google Colab
- Click the 🔑 (Secrets) icon in the left sidebar.  
- Add `SIXTYFOUR_API_KEY` and paste your key.  
- Make sure the notebook can access the secret.


In [2]:

# --- Parameters (you can change these) ---
LEAD_NAME = "Saarth Shah"
LEAD_COMPANY = "Sixtyfour AI"
# Fill as needed below. The more context, the better it performs.
# LEAD_TITLE = "CEO"
# LEAD_LINKEDIN = "https://www.linkedin.com/in/sarah-chen-photography"

BRUTEFORCE = False              # If True, tries common patterns + validation
ONLY_COMPANY_EMAILS = True      # If True, filters out personal emails (gmail, yahoo, etc.)

import os, json, requests

API_KEY = os.getenv("SIXTYFOUR_API_KEY", "")
try:
    from google.colab import userdata
    secret_val = userdata.get("SIXTYFOUR_API_KEY")
    if secret_val:
        os.environ["SIXTYFOUR_API_KEY"] = secret_val
        API_KEY = secret_val
except Exception:
    pass

BASE_URL = "https://api.sixtyfour.ai"
ENDPOINT = f"{BASE_URL}/find-email"
TIMEOUT_S = 180

print("✅ Setup complete (key detected, endpoint ready).")


✅ Setup complete (key detected, endpoint ready).



## Find_Email Request

For most use cases, `find-email` is the perfect tool. You send a request and get an email back with a single call.

The request body requires:

- **lead**: Lead information (name/company recommended; LinkedIn/domain/email if available).

**Optional fields:**

- **bruteforce (bool)**: Whether to try common patterns and validate.
- **only_company_emails (bool)**: If `true`, only company-domain emails are returned.

Responses may take a moment depending on validation steps.


In [4]:

lead = {
    "name": LEAD_NAME,
    "company": LEAD_COMPANY,
    # "title": LEAD_TITLE,
    # "linkedin": LEAD_LINKEDIN,
}

payload = {
    "lead": lead,
    "bruteforce": BRUTEFORCE,
    "only_company_emails": ONLY_COMPANY_EMAILS,
}

resp = requests.post(
    ENDPOINT,
    headers={"x-api-key": API_KEY, "Content-Type": "application/json"},
    json=payload,
    timeout=TIMEOUT_S,
)

if resp.ok:
    data = resp.json()
    print("✅ Success! Keys:", list(data.keys()))
    print("\nResponse:")
    print(json.dumps(data, indent=2))
    # Email field is a list of tuples: [ [email, status, type], ... ]
    emails = data.get("email", [])
    print("\nParsed emails:")
    for item in emails:
        try:
            addr, status, kind = item
            print(f" - {addr} | status={status} | type={kind}")
        except Exception:
            print(" -", item)
else:
    print("❌ Request failed:", resp.status_code, resp.text[:800])


✅ Success! Keys: ['name', 'company', 'email']

Response:
{
  "name": "Saarth Shah",
  "company": "Sixtyfour AI",
  "email": [
    [
      "saarth@sixtyfour.ai",
      "UNKNOWN",
      "COMPANY"
    ]
  ]
}

Parsed emails:
 - saarth@sixtyfour.ai | status=UNKNOWN | type=COMPANY



### How to change values

- Edit the lead fields at the top (name/company/title/LinkedIn).  
- Toggle `BRUTEFORCE` to try common patterns + validation.  
- Toggle `ONLY_COMPANY_EMAILS` to restrict to company-domain addresses.  
- If `lead` already includes an `email`, the API returns it unchanged (status `"OK"`).
- Provide context — More details = better results
